In [ ]:
import pandas as pd
from darts.models import NBEATSModel

Build or load a 'CatchmentData' instance to allow access to all needed data

In [ ]:
rebuild_catchment = False

In [ ]:
from forecasting.catchment_data import CatchmentData
import pickle

if rebuild_catchment:
    catchment = CatchmentData("illinois-kerby", "14377100")

    pickle_out = open("data/catchment.pickle", "wb")
    pickle.dump(catchment, pickle_out)
    pickle_out.close()
else:
    pickle_in = open("data/catchment.pickle", "rb")
    catchment = pickle.load(pickle_in)

In [ ]:
from forecasting.dataset import Dataset
ds = Dataset(catchment)   

In [ ]:
ds.Xs_current[0].pd_dataframe()

Perform a gridsearch to identify the best parameters

In [ ]:
perform_gridsearch = False

In [ ]:
if perform_gridsearch:
    from forecasting.dataset import Dataset
    ds = Dataset(catchment)


    from forecasting.dataset import Dataset
    dataset = Dataset(catchment)

    gridsearch_params = {
        "input_chunk_length" : [120],
        "output_chunk_length" : [96],
        "hidden_size" : [25, 50],
        "n_rnn_layers" : [3, 5, 7],
        "dropout" : [0.0],
        "model" : ["LSTM"]
    }

    X_train = dataset.X_validations[0]
    y_train = dataset.y_validation

    y_train, y_val = y_train.split_before(0.7)

    model,  = NBeatsModel.gridsearch(gridsearch_params, verbose=True, series=y_train, past_covariates=X_train, val_series=y_val)
else:
    # Manually entered best parameters from prior gridsearch
    best_params = {'input_chunk_length': 120, 
    'output_chunk_length': 96}



In [ ]:
overwrite_existing_models = True

In [ ]:
from forecasting.forecaster import Forecaster

forecaster = Forecaster(catchment, 
                                model_type=NBEATSModel, 
                                model_params=best_params, 
                                model_save_dir="NBeats",
                                overwrite_existing_models=overwrite_existing_models)

Fit models for all data in catchment

In [ ]:
forecaster.fit(epochs=20)

Fetch historical forecasts using the trained model. This will generate a forecast for 48 hours into the future every 6 hours for the entire validation set.

In [ ]:
hst_fcasts = forecaster.historical_forecasts(forecast_horizon=48, stride=6, num_samples=100)

In [ ]:
y_pred = hst_fcasts

In [ ]:
y_pred

In [ ]:
y_true = forecaster.dataset.y_test
target_scaler = forecaster.dataset.target_scaler
y_true = target_scaler.inverse_transform(y_true)
y_true = y_true.pd_dataframe()

In [ ]:
pd.concat([y_true, y_pred], join='inner', axis=1)

In [ ]:
start = pd.to_datetime("2013-02-02T21:00:00.000000000")
end = pd.to_datetime("2013-04-01T21:00:00.000000000")
y_pred.loc[start:end, :]['max'].max()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


fig = plt.figure()
plt.plot(y_pred['max'], color='red', linewidth=0.5, label='max 0.05 confidence quantile')
plt.plot(y_pred['mean'], color='red', linewidth=2, label='Predicted value')
plt.plot(y_pred['min'], color='red', linewidth=0.5, label='min 0.05 confidence quantile')

plt.fill_between(y_pred.index, y1=y_pred['min'], y2=y_pred['max'], color='red', alpha=0.3)

plt.plot(y_true.index, y_true['level'],  color='blue', linewidth=2, label='Actual value')
plt.legend()

y_lim = y_pred.loc[start:end, :]['max'].max() * 1.2
plt.xlim(start,end)
plt.ylim(0, y_lim)
plt.show()